In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_41/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# <h1><center><ins>PIPELINE</ins></center></h1>
<h1><center>Checkpoint:</center></h1>
<img src="img/00_pipelines.jpg" alt="Drawing" style="width: 400px;"/>

<a id="tabla_contenidos"></a> 
## Tabla de Contenidos

### <a href='#section_objetivos'>0. Objetivos de la Notebook</a>


### <a href='#section_repaso'>1. Lo que tenés que hacer es...</a>
- #### <a href='#section_reg_vs_clas'>1.1 ...cargar el dataset y preparar los datos:</a>
- #### <a href='#1.2'>1.2 ...aplicar el Pipeline+GridSearchCV:</a>

<a id="section_objetivos"></a> 
## 0. Objetivos de la Notebook

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Poner en práctica lo aprendido en la notebook de práctica guiada.</label>
  <div style="float:left;width: 85%;">
      <label>Es importante que antes de la clase resuelvan esta notebook ya que es fundamental que sepan utilizar las herramientas que vimos en la práctica guiada para después trabajar el caso práctico en la clase.</label>        
</div>    
</div>

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="section_repaso"></a> 
## 1. Lo que tenés que hacer es...

<a id="section_reg_vs_clas"></a>
### 1.1 ...cargar el dataset y preparar los datos:

Vamos a probar cómo armar `pipeline` combinado con `GridSearchCV`. Para ello vamos a volver a utilizar el dataset sobre la calidad de vinos que puede encontrarse en la página de Kaggle: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

El dataset está asociado a variantes rojas de un vino Portuguez. En la página de kaggle pueden encontrar la descripción de qué significan todas las variables. 

Importemos y exploremos un poco el dataset: 

In [2]:
import pandas as pd

df = pd.read_csv('../Data/winequality-red.csv',sep=';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Binarizamos la calidad del vino (columna quality) tomando como criterio arbitrario, todos los vinos que tengan una calidad igual o mayor a 6 los vamos a considerar de buena calidad (les pondemos un 1 como etiqueta), mientras que los que sean inferiores a 6 los consideraremos de baja calidad (y los etiquetaremos con un 0):

In [3]:
df['quality_bin'] = df['quality'].apply(lambda x: 0 if x < 6 else 1)


Separar la `X` (todas las features menos **quality** ni **quality_bin**) y la `y` (**quality_bin**), y separar los datos en aquellos que se van a usar para el entrenamiento y los que servirán para testear el modelo.

In [4]:
from sklearn.model_selection import train_test_split



In [5]:
X = df.drop(['quality_bin','quality'], axis = 1 )

y = df.quality_bin

X_train, X_test, y_train, y_test = train_test_split (X, y, stratify = y, random_state = 1203 )

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="1.2"></a>
### 1.2 ...aplicar el Pipeline+GridSearchCV:

Ahora que los datos están preparados, utilizando `Pipeline`+`GridSearchCV`, armar un proceso general en el que:

**a-** se seleccionen todas las features menos **residual sugar**

**b-** se normalicen los datos

**c-** se aplique una `LogistiRegression`


Con el `GridSearchCV`, el objetivo es explorar si el: en el primer paso **(a)** es mejor seleccionar o no hacer la selección; en el segundo paso **(b)** cuál es el mejor escalador si `StandardScaler` o `MinMaxScaler`, y por último en el paso **(c)** cuál es el valor óptimo del hiperparámetro C de la `LogisticRegression`. 

**(i)** Crear la clase para seleccionar las features:

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

In [7]:
# Creo la clase que escoje entre mis features
class Feature_selector (BaseEstimator, TransformerMixin):
    def __init__(self, selected_feature):
        self.selected_feature = selected_feature
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y= None):
        return X[self.selected_feature]
        


In [8]:
selector = Feature_selector(selected_feature=X.drop(columns='residual sugar').columns.values)
selector

Feature_selector(selected_feature=array(['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'], dtype=object))

**(ii)** Armar los pasos para el `Pipeline` e instancia el `Pipeline`

In [9]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [10]:
steps = [('selecciono_features', selector),
         ('aplico_estandarizacion', StandardScaler()),
         ('estimador', LogisticRegression(solver='lbfgs'))]

pipe = Pipeline(steps)

**(iii)** Generar la grilla y aplicar `GridSearchCV` con la `LogisticRegression`

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [12]:
param_grid = [{'selecciono_features':[selector, None]},
              {'aplico_estandarizacion': [StandardScaler(), MinMaxScaler()]},
              {'estimador__C': np.linspace(0.1,1000,100)}]

In [13]:
gs = GridSearchCV(pipe, param_grid, n_jobs=-1, cv= folds)

In [14]:
gs.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('selecciono_features',
                                        Feature_selector(selected_feature=array(['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'], dtype=object))),
                                       ('aplico_estandarizacion',
                                        Standard...
       6.061e+02, 6.162e+02, 6.263e+02, 6.364e+02, 6.465e+02, 6.566e+02,
       6.667e+02, 6.768e+02, 6.869e+02, 6.970e+02, 7.071e+02, 7.172e+02,
       7.273e+02, 7.374e+02, 7.475e+02, 7.576e+02, 7.677e+02, 7.778e+02,
       7.879e+02, 7.980e+02, 8.081e+02, 8.182e+02, 8.283e+02, 8.384e+02,
       8.485e+02, 8.586e+02, 8.687e+02, 8.788e+02, 8.889e+02, 8.990e+02,
       9.091e+02, 9.192e+02, 9.293e+02, 9.394e+02, 9.495e+02, 9.596e+02,
       9.697e+02, 9.798e+02, 9.899e+02,

In [22]:
gs.best_estimator_

Pipeline(steps=[('selecciono_features',
                 Feature_selector(selected_feature=array(['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'], dtype=object))),
                ('aplico_estandarizacion', StandardScaler()),
                ('estimador', LogisticRegression())])

In [16]:
best_regression = gs.best_estimator_

**(iv)** Identificar cuál fue el mejor modelo, cuál fue su score de validación y evaluar su performance con los datos de test

In [21]:
gs.best_params_

{'selecciono_features': Feature_selector(selected_feature=array(['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides',
        'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
        'sulphates', 'alcohol'], dtype=object))}

In [20]:
gs.best_score_

0.7456380753138075

In [18]:
best_regression.score(X_test, y_test)

0.7325

<a href='#tabla_contenidos'>Volver a TOC</a>